For converting text files of embeddings into xl file

In [9]:
import pandas as pd
import re

# Specify the path to your text file
text_file_path = 'tender_dictionary.txt'

# Initialize lists to store the extracted data
array_names = []
array_data = []

# Open the text file and process its contents
with open(text_file_path, 'r') as file:
    lines = file.readlines()

    # Initialize variables to capture Array_name and Array_data
    current_name = None
    current_data = []

    # Iterate through lines in the file
    for line in lines:
        # Check if the line contains Array_name
        name_match = re.match(r'Array_name: (\w+)', line)
        if name_match:
            # If we already have data from a previous Array_data section, append it
            if current_name is not None:
                array_names.append(current_name)
                array_data.append(current_data)
            
            # Set the current Array_name and reset data
            current_name = name_match.group(1)
            current_data = []
        elif current_name is not None:
            # Check if the line contains numeric data
            data_match = re.findall(r'([-+]?\d*\.\d+e[+-]?\d+|\d+)', line)
            if data_match:
                # Convert scientific notation strings to floating-point numbers
                current_data.extend([float(num) for num in data_match])

    # Append the last set of data if any
    if current_name is not None:
        array_names.append(current_name)
        array_data.append(current_data)

# Create a DataFrame
df = pd.DataFrame({'Array_name': array_names, 'Array_data': array_data})

# Save the DataFrame to an Excel file
excel_file_path = 'output_data.xlsx'
df.to_excel(excel_file_path, index=False)

print(f'Data saved to {excel_file_path}')


Data saved to output_data.xlsx


Updating tenders reference number into embeddings excel sheet 

In [14]:
import pandas as pd

# Load the Excel file without a header
file_path = 'output_data.xlsx'
df = pd.read_excel(file_path, header=0)

# Load the text file and convert it into a list
text_file_path = 'tender_array.txt'
with open(text_file_path, 'r') as text_file:
    new_column_values = eval(text_file.read())  # Assuming your text file contains a valid Python list

# Ensure the lengths match or handle any mismatches
if len(new_column_values) == len(df):
    df.iloc[:, 0] = new_column_values
else:
    # Handle the mismatch (e.g., truncate or fill with default values)
    # Example: Truncate the list to match the DataFrame length
    df.iloc[:, 0] = new_column_values[:len(df)]

# Save the updated DataFrame back to the Excel file
df.to_excel(file_path, index=False, header=None)

print("Column values updated and file saved.")


Column values updated and file saved.


In [18]:
import pandas as pd


file1_path = 'output_data.xlsx'
df1 = pd.read_excel(file1_path)

# Load the second Excel file with 423 rows and header row
file2_path = 'Tenders_WA_descriptionExtracted.xlsx'
df2 = pd.read_excel(file2_path, header=0)

# Perform a left join to keep all rows from df2 and matching rows from df1
merged_df = pd.merge(df2, df1, on='Reference Number', how='inner')

# Save the merged DataFrame to a new Excel file
merged_file_path = 'merged_file.xlsx'
merged_df.to_excel(merged_file_path, index=False)


As we have 1000 records with 424 tenders , some of the tenders are repeating. hence the cluster isn't efficient.
Taking the unique tenders to create cluster.

In [25]:
import pandas as pd

# Load the Excel file into a DataFrame
file_path = 'merged_file.xlsx'
df = pd.read_excel(file_path)

# Specify the column containing the reference numbers (replace 'Reference_Column' with the actual column name)
reference_column_name = 'Reference Number'

# Extract unique reference numbers along with all columns
unique_references_df = df.drop_duplicates(subset=reference_column_name, keep='first')

# Save the DataFrame with unique reference numbers to a new Excel file
unique_references_file_path = 'unique_references.xlsx'
unique_references_df.to_excel(unique_references_file_path, index=False)


416

In [24]:
file_path = 'output_data.xlsx'
df = pd.read_excel(file_path)


column_name = 'Reference Number'

# Count the number of unique values in the specified column
unique_count = df[column_name].nunique()
unique_count

424

Noticed some of the tenders are missing ,so printing the tenders that are missing

In [27]:
import pandas as pd

file1_path = 'unique_references.xlsx'
df1 = pd.read_excel(file1_path)

file2_path = 'output_data.xlsx'
df2 = pd.read_excel(file2_path)

reference_column_name = 'Reference Number'

# Extract the tender reference numbers from both DataFrames
tenders_416 = set(df1[reference_column_name].tolist())
tenders_424 = set(df2[reference_column_name].tolist())

missing_tenders = tenders_424 - tenders_416

# Print the missing tenders
print("Tenders missing in the Excel sheet with 416 records:")
for tender in missing_tenders:
    print(tender)


Tenders missing in the Excel sheet with 416 records:
WAPOL04723
WAPOL0
DOC202246435
FINW0144423
DWER101323
HSS605220
ED230060
ED229594
